# Distance Between 2 Cities the Haversine Formula

### Initialize Data Frames

In [1]:
# Imports Dependencies
import pandas as pd
import numpy as np

In [2]:
def gc_dist_np(lat1, lon1, lat2, lon2):
    """
    Using the Haversine Formula and presuming a spherical Earth with radius R this function  
        calculates the great circle distance (elevation not taken into account) between two points.
    
    Source: https://www.movable-type.co.uk/scripts/gis-faq-5.1.html
    Note: this method is not extremely accurate at short distances (<10km)
    
    Attributes:
        lat1 - Latitude of city 1
        lon1 - Longitude of city 1
        lat2 - Latitude of city 2
        lon2 - Longitude of city 2
        
    Returns:
        Distance in miles
    """
    R = 3956 # Approximate radius of the earth in statute miles
    
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a)) # great circle distance in radians

    return R * c

In [3]:
openings_df = pd.read_table('openings_by_city_lat_long.csv', sep=',')

In [4]:
openings_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
openings_df.rename(columns={"long": "lon"})

,city,openings,lat,lon
0,"New York, NY",169,40.712775,-74.005973
1,"Boston, MA",158,42.360083,-71.058880
2,"Charlotte, NC",106,35.227087,-80.843127
3,"Philadelphia, PA",102,39.952584,-75.165222
4,"Irving, TX",101,32.814018,-96.948894
5,"Yakima, WA",99,46.602071,-120.505899
6,"San Francisco, CA",60,37.774929,-122.419415
7,"Arlington, VA",48,38.881621,-77.090981
8,"Chicago, IL",31,41.878114,-87.629798
9,"Conshohocken, PA",23,40.079277,-75.301571


In [6]:
city_state_df = openings_df['city'].str.split(", ", n = 1, expand = True)
openings_df['city'] = city_state_df[0]
openings_df['state'] = city_state_df[1]

In [7]:
openings_df.sort_values(by=['state', 'lat'], inplace=True)

In [8]:
openings_df.head()

,city,openings,lat,long,state
280,Birmingham,1,33.518589,-86.810357,AL
75,Huntsville,3,34.730369,-86.586104,AL
243,Little Rock,1,34.746481,-92.289595,AR
72,Conway,3,35.088696,-92.442101,AR
207,Fort Huachuca,1,31.555216,-110.349922,AZ


In [10]:
openings_df['dist (m)'] = gc_dist_np(openings_df.lat, \
                            openings_df.long, \
                            openings_df.lat.shift(), \
                            openings_df.long.shift())

In [ ]:
# city_lat_long_df['next_lat'] = city_lat_long_df.lat.shift()
# city_lat_long_df['next_long'] = city_lat_long_df.long.shift()
# city_lat_long_df['dist (m)'] = great_circle_distance_np(city_lat_long_df.lat, \
#                                 city_lat_long_df.long, city_lat_long_df.next_lat, city_lat_long_df.next_long)

In [11]:
openings_df

,city,openings,lat,long,state,dist (m)
280,Birmingham,1,33.518589,-86.810357,AL,NaN
75,Huntsville,3,34.730369,-86.586104,AL,84.643629
243,Little Rock,1,34.746481,-92.289595,AR,323.567439
72,Conway,3,35.088696,-92.442101,AR,25.156460
207,Fort Huachuca,1,31.555216,-110.349922,AZ,1059.977704
56,Tempe,3,33.425510,-111.940005,AZ,158.900808
20,Phoenix,11,33.448377,-112.074037,AZ,7.882354
31,San Diego,6,32.715738,-117.161084,CA,298.580131
131,Poway,2,32.962823,-117.035865,CA,18.542197
45,Carlsbad,4,33.158093,-117.350594,CA,22.659723


In [ ]:
distance_bins = [0, 50]
group_names = ['metro']
city_lat_long_df['distance_bins'] = pd.cut(city_lat_long_df['dist'], distance_bins, labels = group_names)
city_lat_long_df

# District Summary

### Calculate District Summary Values

In [ ]:
# Calculate number of schools in District
district_schools_count = len(school_data.school_name.unique())

# Calculate total student count in District
district_student_total = student_data.student_name.count()

# Calculate total district budget
district_budget = school_data.budget.sum()

# Calculate average math and reading scores throught District
district_avg_math = student_data.math_score.mean()
district_avg_reading = student_data.reading_score.mean()

# Calculate percentage of students who scored 70% or better
district_pass_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
district_pass_math_percent = (district_pass_math / district_student_total) * 100

district_pass_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
district_pass_reading_percent = (district_pass_reading / district_student_total) * 100

# Calculate the average test score across the District
district_overall_pass_percent = (district_avg_math + district_avg_reading) / 2

### Display District Summary

In [ ]:
# Build dict to store District summary data calculated above
district_summary = pd.DataFrame({    
    'Total Schools': [district_schools_count],
    'Total Students': [district_student_total],
    'Total Budget': [district_budget],
    'Average Math Score': [district_avg_math],
    'Average Reading Score': [district_avg_math],
    '% Passing Math': [district_pass_math_percent],
    '% Passing Reading':[district_pass_reading_percent],
    '% Overall Passing Rate': [district_overall_pass_percent]})

# Display District summary data to terminal
district_summary

# School Summary

### Calculate School Summary Values

In [ ]:
# Group merged dataframe by school name & set dataframe index to school name
grouped_by_school = school_data_complete.set_index('school_name').groupby('school_name')

# Determine School types
school_types = school_data.set_index('school_name')['type']

# Calculate students per School
school_student_count = grouped_by_school['Student ID'].count()

# Determine each Schools budget
school_budget = school_data.set_index('school_name')['budget']

# Calculate budget per student
school_budget_per_student = school_data.set_index('school_name')['budget'] /\
                            school_data.set_index('school_name')['size']

# Calculate average math & reading scores by School
school_avg_math = grouped_by_school['math_score'].mean()
school_avg_reading = grouped_by_school['reading_score'].mean()

# Calculate percent math & reading passing rates by School
school_pass_math_percent = school_data_complete[school_data_complete['math_score'] >= 70].groupby \
                        ('school_name')['Student ID'].count() / school_student_count * 100

school_pass_reading_percent = school_data_complete[school_data_complete['reading_score'] >= 70].groupby \
                        ('school_name')['Student ID'].count() / school_student_count * 100

# Calculate overall passing rate by School
school_pass_overall_percent = (school_pass_math_percent + school_pass_reading_percent) / 2

### Display School Summary

In [ ]:
# Build dict to store School summary data calculated above
school_summary = pd.DataFrame({
    'School Type': school_types,
    'Total Students': school_student_count,
    'Total School Budget': school_budget,
    'Per Student Budget': school_budget_per_student,
    'Average Math Score': school_avg_math,
    'Average Reading Score': school_avg_reading,
    '% Passing Math': school_pass_math_percent,
    '% Passing Reading': school_pass_reading_percent,
    'Overall Passing Rate': school_pass_overall_percent})

# Display District summary data to terminal
school_summary

# Top Performing Schools by Overall Passing Rate

In [ ]:
# Sort School summary dataframe created above on 'Overall Passing Rate' in descending order
school_summary_sorted_desc = school_summary.sort_values('Overall Passing Rate', ascending=False)

In [ ]:
# Display top 5 rows of School summary sorted above
school_summary_sorted_desc.head(5)

# Bottom Performing Schools by Overall Passing Rate

In [ ]:
# Determine the bottom performing 5 Schools by 'Overall Passing Rate' still in descending order & reverse order
bottom_schools_sorted_desc = school_summary_sorted_desc.tail(5)
bottom_schools_sorted_asc = bottom_schools_sorted_desc.sort_values('Overall Passing Rate')

In [ ]:
# Display bottom 5 rows of School summary (re)sorted above
bottom_schools_sorted_asc

# Average Math Scores by Grade

In [ ]:
# Determine Math scores for each grade by school
grade_09_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
grade_10_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
grade_11_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
grade_12_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

In [ ]:
# Build dict to store average math score summary data calculated above
avg_math_score_by_grade = pd.DataFrame({
                        '9th': grade_09_math,
                        '10th': grade_10_math,
                        '11th': grade_11_math,
                        '12th': grade_12_math})

# Print index header
avg_math_score_by_grade.index.name = 'School Name'

# Display average math score by grade data to terminal
avg_math_score_by_grade

# Average Reading Scores by Grade

In [ ]:
# Determine Reading scores for each grade by school
grade_09_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
grade_10_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
grade_11_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
grade_12_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean()

In [ ]:
# Build dict to store average Reading score summary data calculated above
avg_reading_score_by_grade = pd.DataFrame({
                        '9th': grade_09_reading,
                        '10th': grade_10_reading,
                        '11th': grade_11_reading,
                        '12th': grade_12_reading})

# Print index header
avg_reading_score_by_grade.index.name = 'School Name'

# Display average reading score by grade data to terminal
avg_reading_score_by_grade

# Scores by School Spending

In [ ]:
# Spending bins and group names given in starter code
spending_bins = [0, 585, 615, 645, 675]
group_names = ['< $585', '$585 - 615', '$615 - 644', '$645-675']

# Add another series to the combined dataframe created above
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/\
                        school_data_complete['size'], spending_bins, labels = group_names)

# Group updated dataframe by spending bins added above
complete_grouped_by_spending = school_data_complete.groupby('spending_bins')

### Calculate School Spending Summary Data

In [ ]:
# Calculate average math & reading score by spending bin
average_math_score_by_spend = complete_grouped_by_spending['math_score'].mean()
average_reading_score_by_spend = complete_grouped_by_spending['reading_score'].mean()

# Calculate students with passing math grades by spend
passing_math_percent_by_spend = school_data_complete[school_data_complete['math_score'] >= 70]\
                        .groupby('spending_bins')['Student ID'].count() / \
                        complete_grouped_by_spending['Student ID'].count() * 100

# Calculate students with passing reading grades by spend
passing_reading_percent_by_spend = school_data_complete[school_data_complete['reading_score'] >= 70]\
                            .groupby('spending_bins')['Student ID'].count() / \
                            complete_grouped_by_spending['Student ID'].count() * 100

# Calculate overall passing percentage by spend
overall_percent_by_spend = (passing_math_percent_by_spend + passing_reading_percent_by_spend) / 2

### Display School Spending Summary Data

In [ ]:
# Build dict to store school spending summary data calculated above
scores_by_school_spending = pd.DataFrame({
    'Average Math Score': average_math_score_by_spend,
    'Average Reading Score': average_reading_score_by_spend,
    '% Passing Math': passing_math_percent_by_spend,
    '% Passing Reading': passing_reading_percent_by_spend,
    'Overall Passing Rate': overall_percent_by_spend})

# Print index header
scores_by_school_spending.index.name = 'Spending per Student'
scores_by_school_spending = scores_by_school_spending.reindex(group_names)

# Display data to terminal
scores_by_school_spending

# Scores by School Size

In [ ]:
# Spending bins and group names given in starter code
size_bins = [0, 1000, 2000, 5000]
group_names = ['Small (<1000)', 'Medium(1000-2000)', 'Large(2000-5000)']

# Add another series to the combined dataframe created above
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], \
                                    size_bins, labels = group_names)

# Group updated dataframe by spending bins added above
complete_grouped_by_size = school_data_complete.groupby('size_bins')

### Calculate Score by School Size Summary Data

In [ ]:
# Calculate average math & reading score by spending bin
average_math_score_by_size = complete_grouped_by_size['math_score'].mean()
average_reading_score_by_size = complete_grouped_by_size['reading_score'].mean()

# Calculate students with passing math grades by spend
passing_math_percent_by_size = school_data_complete[school_data_complete['math_score'] >= 70]\
                        .groupby('size_bins')['Student ID'].count() / \
                        complete_grouped_by_size['Student ID'].count() * 100

# Calculate students with passing reading grades by spend
passing_reading_percent_by_size = school_data_complete[school_data_complete['reading_score'] >= 70]\
                            .groupby('size_bins')['Student ID'].count() / \
                            complete_grouped_by_size['Student ID'].count() * 100

# Calculate overall passing percentage by spend
overall_percent_by_size = (passing_math_percent_by_size + passing_reading_percent_by_size) / 2

### Display Score by School Size Summary Data

In [ ]:
# Build dict to store school spending summary data calculated above
scores_by_school_size = pd.DataFrame({
    'Average Math Score': average_math_score_by_size,
    'Average Reading Score': average_reading_score_by_size,
    '% Passing Math': passing_math_percent_by_size,
    '% Passing Reading': passing_reading_percent_by_size,
    'Overall Passing Rate': overall_percent_by_size})

# Print index header
scores_by_school_size.index.name = 'School Size'
scores_by_school_size = scores_by_school_size.reindex(group_names)

# Display data to terminal
scores_by_school_size

# Scores by School Type

In [ ]:
# Group update dataframe by spending bins added above
complete_grouped_by_type = school_data_complete.groupby('type')

### Calculate Score by School Type Summary Data

In [ ]:
# Calculate average math & reading score by spending bin
average_math_score_by_type = complete_grouped_by_type['math_score'].mean()
average_reading_score_by_type = complete_grouped_by_type['reading_score'].mean()

# Calculate students with passing math grades by spend
passing_math_percent_by_type = school_data_complete[school_data_complete['math_score'] >= 70]\
                        .groupby('type')['Student ID'].count() / \
                        complete_grouped_by_type['Student ID'].count() * 100

# Calculate students with passing reading grades by spend
passing_reading_percent_by_type = school_data_complete[school_data_complete['reading_score'] >= 70]\
                            .groupby('type')['Student ID'].count() / \
                            complete_grouped_by_type['Student ID'].count() * 100

# Calculate overall passing percentage by spend
overall_percent_by_type = (passing_math_percent_by_type + passing_reading_percent_by_type) / 2

### Display Score by School Type Summary Data

In [ ]:
# Build dict to store school spending summary data calculated above
scores_by_school_type = pd.DataFrame({
    'Average Math Score': average_math_score_by_type,
    'Average Reading Score': average_reading_score_by_type,
    '% Passing Math': passing_math_percent_by_type,
    '% Passing Reading': passing_reading_percent_by_type,
    'Overall Passing Rate': overall_percent_by_type})

# Print index header
scores_by_school_type.index.name = 'School Type'

# Display data to terminal
scores_by_school_type